# 2.1.1 Functions

This is a small introduction to functions from the point of view of functional programming _and_ software engineering. It ends with a discussion about the basic way in which functions are represented in an object-oriented programming language like Scala. 

### References

_[Optional]_ __[Why Functional Programming Matters](https://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf)__ John Hughes. This is a classic paper that motivates the need for functional programming by appealing to software enginerring principles such as modularity. It's written using the Miranda programming language, and it's more an academic paper that undergraduate material. Nevertheless, its reading is highly recommeded. Give it a try if you find some time!

__[Scala book (online)](https://docs.scala-lang.org/overviews/scala-book/introduction.html)__.

- [Pure functions](https://docs.scala-lang.org/overviews/scala-book/pure-functions.html)

## What are (pure) functions?

Functions are computational devices that transform input _values_ into output _values_, and do nothing _else_.

In [1]:
// `add one` function

def addOne(x: Int): Int = 
    x+1


defined function addOne

If we run this function, the only thing that happens is the computation of a new value:

In [2]:
addOne(6)

res1: Int = 7

Functions that do something else, besides returning values, are called _impure_ functions. Functional programming deals only with _pure_, or mathematical, functions.

In [3]:
// An impure function

def impureAdd(input: Int): Int = {
    println("adding one to "+input)
    input + 1
}


defined function impureAdd

If we run this function, we will see an _effect_ in the console (besides the pure computation of `input + 1`): 

In [4]:
impureAdd(6)

adding one to 6


res3: Int = 7

There are many kinds of effects: writing to the console, reading from the keyworkd, reading from a socket, calling a web service, executing a query over the database, etc. Clearly, we need effects if we want our programs to do something useful, so pure functions alone are not enough. We will talk about this later on.



## Functions as modularity devices

Why are functions so important in programming? Because they help us to _modularize_ our code. For instance, let's consider the following programs, which access the following data structure of key-value pairs (we will talk about this structure in detail later on):

In [5]:
val config: Map[String, String] = 
    Map("URL" -> "http://hablapps.com",
        "PORT" -> "8080")

config: Map[String, String] = Map(
  "URL" -> "http://hablapps.com",
  "PORT" -> "8080"
)

Our first program access the configuration data for the value of the "URL" key. If it's not found, then the default value "default.url" is returned (similarly, we will discuss the `match` keyword further in the course).

In [2]:
// Program 1
val url: String = config.get("URL") match {
  case Some(u) => u
  case None => "default.url"
}

url: String = "http://hablapps.com"

Our second program accesses the configuration data for the value of the "PORT" key. If it's not found, then the default value "8080" is returned.

In [3]:
// Program 2
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}

port: String = "8080"

These two programs do _almost_ the same. The only differences lie in the particular keys and default values the programs refer to, but, otherwise, they do the same thing. However, this _common factor_ is not reflected in the code. Indeed, we may get one program from the other by copy-pasting, a clear signal of [code-smell](https://en.wikipedia.org/wiki/Code_smell).

These programs are _monolythic_, in the sense that they are not made by composing large enough modules. In this case, the common logic of the program and the values it operates on are intermingled in the same code. 

How can we abstract away the differences and package the common logic in a single module? With functions:

In [6]:

def getKey(config: Map[String, String])(
        key: String, default: String): String = 
    config.get(key) match {
      case Some(p) => p
      case None => default
    }


defined function getKey

This is an abstract module which we can combine with other modules to get back the very same functionality:

In [7]:
// Program 1
/*
val url: String = config.get("URL") match {
  case Some(u) => u
  case None => "default.url"
}
*/

val url: String = getKey(config)("URL", "default.url")

url: String = "http://hablapps.com"

In this case, we combine the module `getKeyFrom` with the modules (data values and variables, in particular) `config`, `"URL"` and `"default.url"`. The composition method is just simple function application.

Which are the advantages of using functions? As in the general case, having a more modular solution enables _reuse_, particularly of those modules which are abstract or parameterised. For instance, we can benefit from this level of reuse by re-implementing the `url` program in the following way:

In [8]:
// Program 2
val port: String = getKey(config)("PORT", "8080")

port: String = "8080"

## Functions as methods

In an object-oriented language, functions are implemented through _methods_, i.e. using the `def` keyword. Note that these methods are invariably part of an `object`, `class` or `trait` declaration. Typically, pure functions are declared as part of objects. For instance, we may declare a set of arithmetic functions as follows: 

In [13]:
import scala.math.{pow, Pi}

class Areas{
    
    def circle(radius: Double): Double = 
        Pi*pow(radius, 2)
    
    def rectangle(width: Double, height: Double): Double = 
        width*height
}

import scala.math.{pow, Pi}


defined class Areas

In [14]:
Areas.circle

cmd14.sc:1: missing argument list for method circle in object Areas
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `circle _` or `circle(_)` instead of `circle`.
val res14 = Areas.circle
                  ^Compilation Failed

: 

In [14]:
(new Areas).circle(6)

res13: Double = 113.09733552923255

In notebooks and the Scala REPL, `def` declarations appear to be independent from any object or class, but they are not:

In [16]:
def foo(i: Int): Int = i
// show errors: "missing arument list for method foo in class Helper"


defined function foo

In [17]:
foo

cmd17.sc:1: missing argument list for method foo in class Helper
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `foo _` or `foo(_)` instead of `foo`.
val res17 = foo
            ^Compilation Failed

: 

When we study higher-order functions, we will see that functions in Scala can also be represented as _objects_, i.e. not only as methods. However, that representation also builds essentially upon methods.

## Functions as values

Functions can also be represented as _values_, i.e. as objects. This allows us to implement functions that receive other functions as arguments, or return functions as results. This special functions are called _higher-order functions_ (HOF), and they feature as a great modularity device. We will mainly discuss this feature of HOFs in PF-3.

In order to represent functions as values, we need first to make extremely clear the difference between variables, values and types. For instance:


In [19]:
var b: Boolean = true

b: Boolean = false

In [21]:
b = false

In [22]:
b

res21: Boolean = false

In [23]:
val b: Boolean = false

b: Boolean = false

In [24]:
b = true

cmd24.sc:1: reassignment to val
val res24 = b = true
              ^Compilation Failed

: 

In [18]:
// three variables
val i: Int = 4
val b: Boolean = true
val s: String = "hola"
val f: <<function type>> = <<lambda expression>>

i: Int = 1
b: Boolean = true
s: String = "hola"

In these definitions, we found three variables: `i`, `s` and `b`. These variables are assigned three __values__: `3`, `"hi"` and `true`. The __types__ of these values are, respectively: `Int`, `String` and `Boolean`. Now, let's consider these other variables that we intend to represent values equivalent to the following function-methods:

In [33]:
// Function-methods

def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

// Function-values

val addOneV: Int => Int = 
    (number: Int) => number + 1

val substractOneV: Int => Int = 
    (number: Int) => number - 1 

defined function addOneM
defined function substractOneM
addOneV: Int => Int = ammonite.$sess.cmd32$Helper$$Lambda$2409/1441529850@5b472363
substractOneV: Int => Int = ammonite.$sess.cmd32$Helper$$Lambda$2410/2078026038@4b56155a

In [34]:
val f1: String => Char = 
    (s: String) => 'a' : Char

f1: String => Char = ammonite.$sess.cmd33$Helper$$Lambda$2413/1981967186@e70d1c4

Here, we also have variables, values and types. The two variables are named `addOneV` and `substractOneV`. They are assigned the _function values_ `(a: Int) => a+1` and `(a: Int) => a - 1`. The type of these values is the same _function type_ `Int => Int`. A function-value is also known as a _lambda expression_. Note that a function value is made of two parts: the input arguments and the function body: `(...input...) => body`. The input arguments declare new variables, each of them of a particular type, that will be assigned to certain values when they are passed to the function (upon invocation). The function body has to be an expression of the type specified as output by the function type.

Function-values are equivalent to function-methods in the sense that they behave exactly in the same way, i.e. they allow us to compute values from other values that we pass as input:

In [35]:
// function-method invocations

addOneM(6)
// equivalent invocations with function-values

addOneV(6)

res34_0: Int = 7
res34_1: Int = 7

But then, which are the advantages of function values? Basically, they allow us to implement HOFs. For instance, let's say that we want to implement a HOF that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and calls this function over a given number. We may want to write something like this:

In [35]:
def call(def int2int(n: Int): Int, number: Int): Int =
    int2int(number)

(console):1:10 expected ")"
def call(def int2int(n: Int): Int, number: Int): Int =
         ^

: 

where the first argument `int2int` attempts to represent any function that receives an integer and returns another integer. 

But this code is not legal in Scala, because arguments to functions need to be values, not methods. That's why we need function-values!

In [36]:
def call(int2int: Int => Int, number: Int): Int =
    int2int(number)

defined function call

which we can use as follows:

In [37]:
// invoke call
call(addOneV : Int => Int, 8 : Int)
call(substractOneV: Int => Int, 4: Int)

res36_0: Int = 9
res36_1: Int = 3

In [39]:
// invoke call
call(addOneV, 8)
call(substractOneV, 4)
val i: Int = 4
call(substractOneV, i)

res38_0: Int = 9
res38_1: Int = 3
i: Int = 4
res38_3: Int = 3

In [41]:
call((e: Int) => e+1 : Int, 5)
call((e: Int) => e*2, 6)

res40_0: Int = 6
res40_1: Int = 12

And we can even pass function-methods that are converted on the fly to function-values!

In [43]:
// invoke call with function-methods
call(addOneV, "a")

cmd43.sc:1: type mismatch;
 found   : String("a")
 required: Int
val res43 = call(addOneV, "a")
                          ^Compilation Failed

: 

In [46]:
val f1: String => Char = 
    (s: String) => 'a' : Char

call(f1: Int => Int, 5)

cmd46.sc:4: type mismatch;
 found   : String => Char
 required: Int => Int
val res46_1 = call(f1: Int => Int, 5)
                   ^Compilation Failed

: 

In [44]:
val d: Char = 'd'


d: Char = 'd'

In [45]:
val i: Int = d

i: Int = 100

In [45]:
val f: Int => Int = f1

cmd45.sc:1: type mismatch;
 found   : String => Char
 required: Int => Int
val f: Int => Int = f1
                    ^Compilation Failed

: 

In [46]:
val f1: Int => Int = (i: Int) => 'a': Char

f1: Int => Int = ammonite.$sess.cmd45$Helper$$Lambda$2453/164140026@778eecf5

In [46]:
val f1: Int => Int = (i: String) => 1

cmd46.sc:1: type mismatch;
 found   : String => Int
 required: Int => Int
val f1: Int => Int = (i: String) => 1
                                 ^Compilation Failed

: 

This conversion is the so-called _eta-expansion_.

In [47]:
def addOneM(i: Int): Int = i+1


defined function addOneM

In [48]:
call(addOneM, 6)

res47: Int = 7

## Syntactic sugar for function-values

We discuss now some syntactic facilities offered by Scala when writing lambda expressions. 

First, we can omit the types of input arguments and let Scala figure out them:

In [52]:
val i = 1

i: Int = 1

In [54]:
val addOneV = 
    (a: Int) => a + 1 : Int

val substractOneV = 
    (a: Int) => a - 1

addOneV: Int => Int = ammonite.$sess.cmd53$Helper$$Lambda$2471/1564571341@23ff40db
substractOneV: Int => Int = ammonite.$sess.cmd53$Helper$$Lambda$2472/1586035199@6c00f532

In [58]:
val addOneV = 
    a => a + 1


cmd58.sc:2: missing parameter type
    a => a + 1
    ^Compilation Failed

: 

In [55]:
val addOneV: Double => Double = 
    a => a + 1


addOneV: Double => Double = ammonite.$sess.cmd54$Helper$$Lambda$2478/114769358@702f95

In [56]:
val addOneV: Short => Double = 
    a => a + 1


addOneV: Short => Double = ammonite.$sess.cmd55$Helper$$Lambda$2480/191003528@2237705a

In [58]:
val addOneV: String => String = 
    a => a + 1


addOneV: String => String = ammonite.$sess.cmd57$Helper$$Lambda$2484/780444825@7f8be87b

In [ ]:
val addOneV: Int => Int = 
    a => a + 1

val substractOneV: Int => Int = 
    a => a - 1

addOneV: Int => Int = ammonite.$sess.cmd49$Helper$$Lambda$2461/1511799979@3b6af50a
substractOneV: Int => Int = ammonite.$sess.cmd49$Helper$$Lambda$2462/1465292080@63b49588

Second, we can get extra level of conciseness using so-called _underscore_ syntax:

In [60]:
val addOne: Int => Int = 
    _ + 1

addOne: Int => Int = ammonite.$sess.cmd59$Helper$$Lambda$2488/1345723566@2db34276

In [61]:
val times2: Int => Int = 
    i => i + i

times2: Int => Int = ammonite.$sess.cmd60$Helper$$Lambda$2490/953066681@26c1b97e

In [61]:
val times2: Int => Int = 
    _ + _

cmd61.sc:2: missing parameter type for expanded function ((<x$1: error>, x$2) => x$1.$plus(x$2))
    _ + _
    ^cmd61.sc:2: missing parameter type for expanded function ((<x$1: error>, <x$2: error>) => x$1.$plus(x$2))
    _ + _
        ^Compilation Failed

: 

In [8]:
// call((a: Int) => a + 1, 5)
// call((a: Int) => a - 1, 3)

## Currying

What about functions that receive more than one argument? We would like to implement the function-value equivalent of this function-method: 

In [62]:
// function-method sum
def sum(x: Int, y: Int): Int = 
    x+y

defined function sum

We do that as follows:

In [64]:
// function-value sum
val sum: (Int, Int) => Int =
    (x: Int, y: Int) => x+y : Int


sum: (Int, Int) => Int = ammonite.$sess.cmd63$Helper$$Lambda$2504/1384813608@22db3175

or, exploiting type inference:

In [65]:
// function-value sum, with type-inference & underscore syntax

val sum: (Int, Int) => Int = 
    (x, y) => x+y

sum: (Int, Int) => Int = ammonite.$sess.cmd64$Helper$$Lambda$2506/140523330@4933c670

In [66]:
// function-value sum, with type-inference & underscore syntax

val sum: (Int, Int) => Int = 
    _+_

sum: (Int, Int) => Int = ammonite.$sess.cmd65$Helper$$Lambda$2508/165123381@446aa71

In [69]:
val f: (Int, String, Boolean, Char, Float) => Long = 
    (i, s, b, c, f) => ???

f: (Int, String, Boolean, Char, Float) => Long = ammonite.$sess.cmd68$Helper$$Lambda$2517/1695366348@ea10d36

In [70]:
f(1, "", true, 'a', 1.0f)

: 

However, function types of two, three, ... arguments are not extrictly necessary, and sometimes we can get along with functions of one argument. But, how can we create a function of two arguments with functions of one argument alone? The trick is the following:

In [72]:
// function-value sum, currified
val f: Int => (Int => Int) = 
    (x: Int) => ??? : (Int => Int)

f: Int => Int => Int = ammonite.$sess.cmd71$Helper$$Lambda$2521/557959504@5f3682a7

In [82]:
val sum: Int => (Int => Int) = 
    (x: Int) => (y: Int) => x+y : Int

sum: Int => Int => Int = ammonite.$sess.cmd81$Helper$$Lambda$2715/1261058419@6ea4f0f5

In [83]:
val sum: Int => (Int => Int) = 
    (x: Int) => y => x+y

sum: Int => Int => Int = ammonite.$sess.cmd82$Helper$$Lambda$2717/1045595003@3632c65

In [84]:
val sum: Int => (Int => Int) = 
    (x: Int) => x+_

sum: Int => Int => Int = ammonite.$sess.cmd83$Helper$$Lambda$2719/1958400184@63e3a13f

In [84]:
val sum: Int => (Int => Int) = 
    _+_

cmd84.sc:2: missing parameter type for expanded function ((<x$1: error>, x$2) => x$1.$plus(x$2))
    _+_
    ^cmd84.sc:2: missing parameter type for expanded function ((<x$1: error>, <x$2: error>) => x$1.$plus(x$2))
    _+_
      ^Compilation Failed

: 

In [85]:
// function-value sum, currified
val sum: Int => (Int => Int) = 
    (x: Int) => (y: Int) => x+y : Int

sum: Int => Int => Int = ammonite.$sess.cmd84$Helper$$Lambda$2721/1847246330@47afbf91

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

In [86]:
// partial application
val addTen: Int => Int = sum(10)
addTen(6)
sum(10)(6)

addTen: Int => Int = ammonite.$sess.cmd84$Helper$$Lambda$2723/1932928281@175b9d1a
res85_1: Int = 16
res85_2: Int = 16

returns a function that can be applied again:

In [8]:
// total application


We can apply this strategy to functions of any number of arguments. This is called _currying_ and _currified functions_. The analog in function-methods is [multiple-parameter lists](https://docs.scala-lang.org/tour/multiple-parameter-lists.html):

In [93]:
val f: (Int => Int) => Int = 
    (g: Int => Int) => g(0) : Int

f: (Int => Int) => Int = ammonite.$sess.cmd92$Helper$$Lambda$2735/1015910170@5f973411

In [87]:
// function-method, with multi-parameter list
val sum: Int => (Int => Int) = x => y => x+y

sum: Int => Int => Int = ammonite.$sess.cmd86$Helper$$Lambda$2727/646764605@92d901b

In [94]:
// function-method, with multi-parameter list
val sum: Int => Int => Int = x => y => x+y

sum: Int => Int => Int = ammonite.$sess.cmd93$Helper$$Lambda$2737/1304382094@371e3a33

In [88]:
def sum(x: Int)(y: Int): Int = x+y

defined function sum

In [89]:
sum(6) _

res88: Int => Int = ammonite.$sess.cmd88$Helper$$Lambda$2729/1297104315@7320e851

In [90]:
val f: Int => Int = sum(6)

f: Int => Int = ammonite.$sess.cmd89$Helper$$Lambda$2731/1528696172@1a0822f3

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [8]:
// isEvenLength


This function is somehow the combination of two more basic functions `length` and `isEven`:

In [8]:
// length 


In [8]:
// isEven


but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` using the functions `length` and `isEven`? We can use a HOF which helps us to compose functions:

In [8]:
// compose HOF


Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [8]:
// val isEvenLength: String => Boolean = ???

The HOF `compose` is actually defined for function types of one argument: 

In [8]:
// val isEvenLength: String => Boolean = ???

or using infix notation:

In [8]:
// val isEvenLength: String => Boolean = ???

Note that a similar function to `compose`, called `andThen`, is also available in the standard library: 

In [8]:
// val isEvenLength: String => Boolean = ???

The last implementation of the `compose` HOF was _monomorphic_, in the sense that it only works with specific types. We can obtain a more flexible implementation using generics, also known as _parametric polymorphism_:

In [9]:
def compose(f2: Int => Boolean, f1: String => Int): String => Boolean = 
    (a: String) => f2(f1(a))

defined function compose

We can also give a currified version of this function as follows:

In [9]:
// compose currified


Last, there is a function which behaves as the identity element with respect to the operation `compose`, i.e. no matter which other function we choose to compose with the [`identity`](https://www.scala-lang.org/api/current/scala/Predef$.html) function, the result will be that function:
1. `identity[B] compose f == f` for all `f: A => B`
2. `f compose identity[A] == f` for all `f: A => B`

In [9]:
// function-method identity


or using lambda expressions:

In [9]:
// quasi function-value identity


## How are functions represented as values

Now, we just want to focus on how are functions actually represented as values in a OO language like Scala. This representation builds essentially upon methods, in particular, _reified_ methods. For instance, let's consider the following functions:

In [10]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

defined function addOneM
defined function substractOneM

In order to create a type of functions that receive an integer and return another one, we can create a new class whose only method is the function that we want to actually implement:

In [10]:
// FunctionInt2Int class 


Now, we can implement the `call` HOF as follows: 

In [10]:
// call HOF


In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [10]:
// addOneV and SubstractV function-values


We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [10]:
// invoke call HOF

Actually, function types such as `Int => Int` and `Boolean => String` are syntactic sugar for the types `Function1[Int, Int]` and `Function1[Boolean, String]`, where [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html) is a generalization of the type `FunctionInt2Int` that we wrote above. We have also [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), [`Function3`](https://www.scala-lang.org/api/current/scala/Function3.html), etc., that are roughly implemented as follows:

In [11]:
object Std{
    trait Function1[A, B]{
        def apply(a: A): B
    }

    trait Function2[A, B, C]{
        def apply(a: A, b: B): C
    }

    // up to Function22
}


defined object Std

Using these standard classes, we can create the `addOneV` function-value in a similar way than before: 

In [11]:
// addOneV as a Function1 instance


and invoke functions as follows:


In [11]:
// invoke addOneV


However, as we saw throughout this notebook, we can also invoke the function without explicitly naming the `apply` method, i.e. 

In [11]:
// omit apply!


This is just another syntactic nicety of Scala. In sum, in an object-oriented language like Scala, function-values are ultimately methods in disguise.